In [1]:
import os
import shutil
import sys
import numpy as np
import scipy
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
import cv2

C:\Users\Sandeep\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_path = os.getcwd() + "\\data\\"

In [3]:
def read(data_path, folder_type='train'):
    if folder_type.lower() == 'train':
        path = data_path + "\\" + "Training"
    elif folder_type.lower() == 'test':
        path = data_path + "\\" + "Test"
    else:
        print("Wrong folder path")
        sys.exit(1)
    
    total_fruits_list  = os.listdir(path)
    total_fruits_count = len(total_fruits_list)
    
    
    
    all_images_count = 0
    
    total_fruits_list_temp = total_fruits_list[:5]
    total_fruits_count_temp = len(total_fruits_list_temp)
    
    for fruit in total_fruits_list_temp:        
        for img in os.listdir(path + "\\" + fruit):
            all_images_count += 1
            
    
    data = {}
    labels = {}
    data[folder_type] = np.zeros(shape=(all_images_count, 100, 100, 3), dtype=np.float64)
    labels[folder_type] = np.zeros(all_images_count)
    print(data[folder_type].shape)
    print(labels[folder_type].shape)
    
    data_counter = 0
    label_counter = 0
    
    label_to_idx_dict = {}
    idx_to_label_dict = {}
    
    for fruit in total_fruits_list_temp:        
        print(folder_type, " : ", fruit)  
        label_to_idx_dict[fruit] = label_counter
        idx_to_label_dict[label_counter] = fruit
        for img in os.listdir(path + "\\" + fruit):
            cur_img = cv2.imread(path + "\\" + fruit + "\\" + img)            
            data[folder_type][data_counter,:,:,:] = cur_img                        
            labels[folder_type][data_counter] = label_counter
            data_counter += 1
        label_counter += 1
            
    return data[folder_type], labels[folder_type], len(total_fruits_list_temp), label_to_idx_dict, idx_to_label_dict
    

In [4]:
training_data, training_labels, num_classes, \
    label_to_idx_dict_train,idx_to_label_dict_train = read(data_path, folder_type="train")

test_data, test_labels, num_classes, \
    label_to_idx_dict_test, idx_to_label_dict_test = read(data_path, folder_type="test")

(2449, 100, 100, 3)
(2449,)
train  :  Apple Braeburn
train  :  Apple Golden 1
train  :  Apple Golden 2
train  :  Apple Golden 3
train  :  Apple Granny Smith
(817, 100, 100, 3)
(817,)
test  :  Apple Braeburn
test  :  Apple Golden 1
test  :  Apple Golden 2
test  :  Apple Golden 3
test  :  Apple Granny Smith


In [ ]:
'''
hf_train_data = h5py.File('training_data.h5', 'w')
hf_train_labels = h5py.File('training_labels.h5', 'w')
hf_test_data = h5py.File('test_data.h5', 'w')
hf_test_labels = h5py.File('test_labels.h5', 'w')

hf_train_data.create_dataset('training_data', data=training_data)
asciiList = [n.encode("ascii", "ignore") for n in training_labels]
hf_train_labels.create_dataset('training_labels', (len(asciiList),1),'S25', asciiList)

hf_test_data.create_dataset('test_data', data=test_data)
asciiList_test = [n.encode("ascii", "ignore") for n in test_labels]
hf_test_labels.create_dataset('test_labels', (len(asciiList_test),1),'S25', asciiList_test)
'''

In [5]:
# Display the dimensions of the data
print("Shape of training data is {}".format(training_data.shape)) 
print("Shape of training labels is {}".format(training_labels.shape)) 

print("Shape of test data is {}".format(test_data.shape)) 
print("Shape of test labels is {}".format(test_labels.shape)) 

Shape of training data is (2449, 100, 100, 3)
Shape of training labels is (2449,)
Shape of test data is (817, 100, 100, 3)
Shape of test labels is (817,)


In [6]:
label_to_idx_dict_train

{'Apple Braeburn': 0,
 'Apple Golden 1': 1,
 'Apple Golden 2': 2,
 'Apple Golden 3': 3,
 'Apple Granny Smith': 4}